# Final Project (Group): "Aerial Threat Detection: Soldier and Civilian Classification Using Drone Vision and Deep Learning"

# INSTALL REQUIRED PACKAGES

In [ ]:
# Install required packages for YOLOv8 training and data augmentation
!pip install -q ultralytics albumentations opencv-python-headless

# Import essential libraries
import os
import shutil
import yaml
import random
import numpy as np
from pathlib import Path
import cv2
from PIL import Image
import albumentations as A
from google.colab import drive

print("All packages installed successfully!")
print("Libraries imported and ready for use")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.9 MB/s eta 0:00:00
All packages installed successfully!
Libraries imported and ready for use


# MOUNT GOOGLE DRIVE

In [ ]:
# Mount Google Drive and Auto-Detect Dataset Structure
from google.colab import drive
import os
import shutil
from pathlib import Path
import random

drive.mount('/content/drive')

print("="*60)
print("STEP 1: Auto-detecting dataset structure...")
print("="*60)

# Updated path to the correct dataset location
GDRIVE_DATASET_PATH = '/content/drive/MyDrive/csc-final-project-aerial.v1-csc_dataset.yolov8'

dataset_source = Path(GDRIVE_DATASET_PATH)
working_dir = '/content/civilian_soldier_working'

print(f"\nSearching in: {dataset_source}")

if not dataset_source.exists():
    print(f"\nERROR: Path not found: {dataset_source}")
    print("\nPlease ensure the dataset folder exists at:")
    print(f"  {GDRIVE_DATASET_PATH}")
    raise FileNotFoundError(f"Dataset path not found: {dataset_source}")

# Show what's in the dataset folder
print(f"\nContents of {dataset_source.name}:")
for item in sorted(dataset_source.iterdir())[:20]:
    if item.is_dir():
        file_count = len(list(item.iterdir()))
        print(f"  {item.name}/ ({file_count} items)")
    else:
        print(f"  {item.name}")

# Auto-detect dataset structure based on data.yaml format
print(f"\n{'='*60}")
print("STEP 2: Detecting YOLO dataset structure...")
print("="*60)

# Check for standard YOLOv8 structure: train/, valid/, test/ folders
train_dir = dataset_source / 'train'
valid_dir = dataset_source / 'valid'
test_dir = dataset_source / 'test'

if train_dir.exists() and valid_dir.exists():
    print(f"\nFound standard YOLOv8 structure:")
    print(f"  train/ - {len(list((train_dir / 'images').glob('*'))) if (train_dir / 'images').exists() else 0} images")
    print(f"  valid/ - {len(list((valid_dir / 'images').glob('*'))) if (valid_dir / 'images').exists() else 0} images")
    if test_dir.exists():
        print(f"  test/ - {len(list((test_dir / 'images').glob('*'))) if (test_dir / 'images').exists() else 0} images")
else:
    print("\nERROR: Expected train/ and valid/ folders not found!")
    raise ValueError("Invalid dataset structure")

# Copy dataset to Colab workspace
print(f"\n{'='*60}")
print("STEP 3: Copying dataset to Colab workspace...")
print("="*60)

os.makedirs(working_dir, exist_ok=True)
working_path = Path(working_dir)

# Copy all splits (train, valid/val, test)
for split in ['train', 'valid', 'test']:
    src_split = dataset_source / split
    if src_split.exists():
        # Normalize split name (valid -> val)
        dest_split = 'val' if split == 'valid' else split

        # Copy images
        src_images = src_split / 'images'
        dst_images = working_path / dest_split / 'images'
        if src_images.exists():
            dst_images.parent.mkdir(exist_ok=True, parents=True)
            if dst_images.exists():
                shutil.rmtree(dst_images)
            shutil.copytree(src_images, dst_images)
            img_count = len(list(dst_images.glob('*')))
            print(f"Copied {split} images: {img_count} files")

        # Copy labels
        src_labels = src_split / 'labels'
        dst_labels = working_path / dest_split / 'labels'
        if src_labels.exists():
            dst_labels.parent.mkdir(exist_ok=True, parents=True)
            if dst_labels.exists():
                shutil.rmtree(dst_labels)
            shutil.copytree(src_labels, dst_labels)
            lbl_count = len(list(dst_labels.glob('*.txt')))
            print(f"Copied {split} labels: {lbl_count} files")

os.chdir(working_dir)

print(f"\n{'='*60}")
print("SUCCESS: Dataset ready!")
print("="*60)
print(f"Working directory: {os.getcwd()}\n")

# Show final structure
print("Final dataset structure:")
for split in ['train', 'val', 'test']:
    split_path = working_path / split
    if split_path.exists():
        print(f"\n{split}/")
        for subdir in sorted(split_path.iterdir()):
            if subdir.is_dir():
                count = len(list(subdir.glob('*')))
                print(f"  {subdir.name}/ ({count} files)")

print(f"\n{'='*60}")
print("Ready for training!")
print("="*60)

Mounted at /content/drive
STEP 1: Auto-detecting dataset structure...

Searching in: /content/drive/MyDrive/csc-final-project-aerial.v1-csc_dataset.yolov8

Contents of csc-final-project-aerial.v1-csc_dataset.yolov8:
  README.dataset.txt
  README.roboflow.txt
  data.yaml
  test/ (2 items)
  train/ (2 items)
  valid/ (2 items)

STEP 2: Detecting YOLO dataset structure...

Found standard YOLOv8 structure:
  train/ - 2825 images
  valid/ - 120 images
  test/ - 52 images

STEP 3: Copying dataset to Colab workspace...
Copied train images: 2825 files
Copied train labels: 2825 files
Copied valid images: 120 files
Copied valid labels: 104 files
Copied test images: 52 files
Copied test labels: 52 files

SUCCESS: Dataset ready!
Working directory: /content/civilian_soldier_working

Final dataset structure:

train/
  images/ (2825 files)
  labels/ (2825 files)

val/
  images/ (120 files)
  labels/ (104 files)

test/
  images/ (52 files)
  labels/ (52 files)

Ready for training!


# CREATE DATASET AUGMENTATION PIPELINE

In [ ]:
def create_augmentation_pipeline():
    """Create advanced augmentation pipeline for musical instrument detection images"""

    # Define 10 different augmentation transforms with bbox support
    transform_set_1 = A.Compose([
        A.RandomRotate90(p=0.7),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.6
        ),
        A.GaussNoise(noise_scale_factor=0.1, p=0.4),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_2 = A.Compose([
        A.VerticalFlip(p=0.3),
        A.RandomBrightnessContrast(
            brightness_limit=0.3,
            contrast_limit=0.3,
            p=0.7
        ),
        A.Blur(blur_limit=3, p=0.3),
        A.RandomGamma(gamma_limit=(80, 120), p=0.4),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_3 = A.Compose([
        A.Rotate(limit=15, p=0.6),
        A.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05, p=0.5),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_4 = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_5 = A.Compose([
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.6),
        A.GaussianBlur(blur_limit=3, sigma_limit=0, p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_6 = A.Compose([
        A.RandomRotate90(p=0.5),
        A.VerticalFlip(p=0.4),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.6),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_7 = A.Compose([
        A.HorizontalFlip(p=0.6),
        A.GaussNoise(noise_scale_factor=0.15, p=0.5),
        A.RandomGamma(gamma_limit=(70, 130), p=0.5),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_8 = A.Compose([
        A.Rotate(limit=20, p=0.7),
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=15, p=0.6),
        A.Blur(blur_limit=4, p=0.4),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_9 = A.Compose([
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.35, contrast_limit=0.35, p=0.6),
        A.ISONoise(color_shift=(0.01, 0.08), intensity=(0.15, 0.6), p=0.4),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    transform_set_10 = A.Compose([
        A.RandomRotate90(p=0.6),
        A.HorizontalFlip(p=0.5),
        A.CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), p=0.5),
        A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.15, hue=0.08, p=0.5),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    return [transform_set_1, transform_set_2, transform_set_3, transform_set_4, transform_set_5,
            transform_set_6, transform_set_7, transform_set_8, transform_set_9, transform_set_10]

def parse_yolo_annotation(annotation_path):
    """Parse YOLO format annotation file"""
    bboxes = []
    class_labels = []

    if annotation_path.exists():
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(float(parts[0]))  # Convert to int, handle floats like '2.0'
                    bbox = [float(x) for x in parts[1:5]]  # x_center, y_center, width, height
                    class_labels.append(class_id)
                    bboxes.append(bbox)

    return bboxes, class_labels

def save_yolo_annotation(annotation_path, bboxes, class_labels):
    """Save YOLO format annotation file"""
    with open(annotation_path, 'w') as f:
        for bbox, class_id in zip(bboxes, class_labels):
            # Format: class_id x_center y_center width height
            f.write(f"{class_id} {' '.join(map(str, bbox))}\n")

# Create augmentation pipelines
augmentation_pipelines = create_augmentation_pipeline()
print("Created 10 augmentation pipelines for civilian/soldier detection:")
print("Pipeline 1: Rotation, flip, brightness, noise")
print("Pipeline 2: Vertical flip, brightness, blur, gamma")
print("Pipeline 3: Rotation, color jitter, ISO noise")
print("Pipeline 4: Horizontal flip, HSV, CLAHE")
print("Pipeline 5: Rotation, brightness, Gaussian blur")
print("Pipeline 6: Rotate90, vertical flip, color jitter")
print("Pipeline 7: Horizontal flip, Gauss noise, gamma")
print("Pipeline 8: Rotation, HSV, blur")
print("Pipeline 9: Vertical flip, brightness, ISO noise")
print("Pipeline 10: Rotate90, horizontal flip, CLAHE, color jitter")
print("This will generate 10x more training data for better person detection!")

Created 10 augmentation pipelines for civilian/soldier detection:
Pipeline 1: Rotation, flip, brightness, noise
Pipeline 2: Vertical flip, brightness, blur, gamma
Pipeline 3: Rotation, color jitter, ISO noise
Pipeline 4: Horizontal flip, HSV, CLAHE
Pipeline 5: Rotation, brightness, Gaussian blur
Pipeline 6: Rotate90, vertical flip, color jitter
Pipeline 7: Horizontal flip, Gauss noise, gamma
Pipeline 8: Rotation, HSV, blur
Pipeline 9: Vertical flip, brightness, ISO noise
Pipeline 10: Rotate90, horizontal flip, CLAHE, color jitter
This will generate 10x more training data for better person detection!


/usr/local/lib/python3.12/dist-packages/albumentations/augmentations/blur/functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 4) automatically adjusted to (3, 5).
  result = _ensure_odd_values(result, info.field_name)


# YOLO CONFIGURATION FILE

In [ ]:
def create_yolo_config():
    """Create YAML configuration for YOLOv8 training"""
    print("⚙️ Creating YOLO configuration file...")

    # Updated to use the correct dataset path
    custom_yaml_path = Path(GDRIVE_DATASET_PATH) / 'data.yaml' if 'GDRIVE_DATASET_PATH' in globals() else None

    if custom_yaml_path is not None and custom_yaml_path.exists():
        print("Found data.yaml file - using it!")

        # Read existing data.yaml
        with open(custom_yaml_path, 'r') as f:
            config = yaml.safe_load(f)

        # Update paths to working directory
        config['path'] = str(Path.cwd())
        config['train'] = 'train/images'
        config['val'] = 'val/images'
        config['test'] = 'test/images'

        # Ensure we have 2 classes (civilian and soldier)
        config['nc'] = 2
        config['names'] = ['civilian', 'soldier']

        # Save updated config as dataset.yaml
        with open('dataset.yaml', 'w') as f:
            yaml.dump(config, f, default_flow_style=False)

        print("YOLO configuration created successfully!")
        print(f"Dataset path: {config['path']}")
        print(f"Number of classes: {config['nc']}")
        print(f"Class names: {config['names']}")
    else:
        print("data.yaml not found - creating default configuration")

        config = {
            'path': str(Path.cwd()),
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'nc': 2,
            'names': ['civilian', 'soldier']
        }

        with open('dataset.yaml', 'w') as f:
            yaml.dump(config, f, default_flow_style=False)

        print("YOLO configuration created successfully!")
        print(f"Dataset path: {config['path']}")
        print(f"Number of classes: {config['nc']}")
        print(f"Class names: {config['names']}")

    print("\nFinal Dataset Summary:")
    for split in ['train', 'val', 'test']:
        images_dir = Path(split) / 'images'
        labels_dir = Path(split) / 'labels'
        if images_dir.exists() and labels_dir.exists():
            image_count = len(list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.png')))
            label_count = len(list(labels_dir.glob('*.txt')))
            print(f"  {split}: {image_count} images, {label_count} labels")

    return config

dataset_config = create_yolo_config()

print("\nReady for YOLOv8 training!")

⚙️ Creating YOLO configuration file...
Found data.yaml file - using it!
YOLO configuration created successfully!
Dataset path: /content/civilian_soldier_working
Number of classes: 2
Class names: ['civilian', 'soldier']

Final Dataset Summary:
  train: 2825 images, 2825 labels
  val: 120 images, 104 labels
  test: 52 images, 52 labels

Ready for YOLOv8 training!


# TRAIN YOLOv8 MODEL

In [ ]:
from ultralytics import YOLO
import torch

print("🚁 Starting YOLOv8 training on civilian/soldier aerial detection dataset...")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA Available: {torch.cuda.is_available()}")

# Initialize YOLOv8 model
print("Loading YOLOv8n model...")
model = YOLO('yolov8n.pt')  # Using nano model for faster training

# Start training with optimized parameters
print("Starting training with optimized hyperparameters...")

results = model.train(
    data='dataset.yaml',           # Dataset configuration file
    epochs=100,                    # Number of training epochs
    patience=15,                   # Early stopping patience
    batch=8,                       # Batch size (adjust based on GPU memory)
    imgsz=640,                     # Image size for training
    device=0 if torch.cuda.is_available() else 'cpu',  # Use GPU if available
    project='runs/train',          # Project directory for saving results
    name='custom_aerial_detection',# Experiment name
    save=True,                     # Save model checkpoints
    save_period=-1,                # Save checkpoint every N epochs (-1 = only last)
    cache=False,                   # Cache images for faster training
    workers=8,                     # Number of dataloader workers
    optimizer='auto',              # Optimizer
    verbose=True,                  # Print verbose output
    seed=0,                        # Random seed for reproducibility
    deterministic=True,            # Deterministic mode
    single_cls=False,              # Train as multi-class
    rect=False,                    # Rectangular training
    cos_lr=False,                  # Cosine learning rate scheduler
    close_mosaic=10,               # Disable mosaic augmentation in last N epochs
    resume=False,                  # Resume from last checkpoint
    amp=True,                      # Automatic Mixed Precision
    fraction=1.0,                  # Dataset fraction to use
    profile=False,                 # Profile ONNX and TensorRT speeds
    freeze=None,                   # Freeze layers
    plots=True,                    # Save training plots
    val=True                       # Validate after training
)

print("\n✅ Training completed successfully!")
print(f"📁 Training results saved to runs/train/custom_aerial_detection")

# Store results for metrics calculation
training_results = results

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚁 Starting YOLOv8 training on civilian/soldier aerial detection dataset...
Device: NVIDIA A100-SXM4-40GB
CUDA Available: True
Loading YOLOv8n model...
Starting training with optimized hyperparameters...
Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, e

#UPLOAD RESULTS TO GOOGLE DRIVE

In [ ]:
def upload_results_to_drive():
    """Upload all training results back to Google Drive"""

    print("☁️ Uploading results to Google Drive...")

    # Define Google Drive destination
    drive_destination = '/content/drive/MyDrive/csc-final-project-aerial.v1-csc_dataset.yolov8/trained_models'

    # Create destination directory
    os.makedirs(drive_destination, exist_ok=True)

    # Find the latest training run
    runs_dir = Path('runs/train')
    if runs_dir.exists():
        latest_run = max(runs_dir.glob('custom_aerial_detection*'), key=lambda x: x.stat().st_mtime)
        weights_dir = latest_run / 'weights'

        if weights_dir.exists():
            # Upload model files
            model_files = ['best.pt', 'last.pt']

            for model_file in model_files:
                src_path = weights_dir / model_file
                if src_path.exists():
                    dst_path = f"{drive_destination}/{model_file}"
                    shutil.copy2(src_path, dst_path)
                    size_mb = src_path.stat().st_size / 1024 / 1024
                    print(f"✅ Uploaded {model_file} ({size_mb:.1f} MB)")

            # Upload training plots and results
            result_files = [
                'results.png',
                'results.csv',
                'confusion_matrix.png',
                'confusion_matrix_normalized.png',
                'F1_curve.png',
                'P_curve.png',
                'R_curve.png',
                'PR_curve.png',
                'BoxF1_curve.png',
                'BoxPR_curve.png',
                'BoxP_curve.png',
                'BoxR_curve.png',
                'labels.jpg',
                'labels_correlogram.jpg',
                'val_batch0_labels.jpg',
                'val_batch0_pred.jpg',
                'val_batch1_labels.jpg',
                'val_batch1_pred.jpg',
                'val_batch2_labels.jpg',
                'val_batch2_pred.jpg',
                'train_batch0.jpg',
                'train_batch1.jpg',
                'train_batch2.jpg',
                'args.yaml'
            ]

            for result_file in result_files:
                src_path = latest_run / result_file
                if src_path.exists():
                    dst_path = f"{drive_destination}/{result_file}"
                    shutil.copy2(src_path, dst_path)
                    print(f"✅ Uploaded {result_file}")

    # Upload dataset configuration
    if Path('dataset.yaml').exists():
        dst_path = f"{drive_destination}/dataset.yaml"
        shutil.copy2('dataset.yaml', dst_path)
        print("✅ Uploaded dataset configuration")

    print(f"\n📁 All files uploaded to: {drive_destination}")
    print("🔗 Access your trained models directly from Google Drive!")

    return drive_destination

# Upload results to Google Drive
drive_path = upload_results_to_drive()

# Prepare download summary
print("\n📥 Files ready for download:")

files_to_download = [
    'runs/train/custom_aerial_detection/weights/best.pt',
    'runs/train/custom_aerial_detection/weights/last.pt',
    'runs/train/custom_aerial_detection/results.png',
    'runs/train/custom_aerial_detection/confusion_matrix.png',
]

for file_path in files_to_download:
    if Path(file_path).exists():
        size = Path(file_path).stat().st_size / 1024 / 1024
        print(f"  ✅ {file_path} ({size:.1f} MB)")
    else:
        print(f"  ❌ {file_path} (not found)")

# Display final success message
print(f"""
╔══════════════════════════════════════════════════════════════╗
║    🚁 AERIAL THREAT DETECTION TRAINING COMPLETE! 🚁          ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  🎯 Model: YOLOv8n                                          ║
║  📊 Classes: Civilian & Soldier                             ║
║  🖼️  Image Size: 640x640                                    ║
║  🔄 Training Complete                                        ║
║                                                              ║
║  📥 Downloads Available:                                     ║
║    • best.pt (trained model)                                ║
║    • last.pt (latest checkpoint)                            ║
║    • Training plots and metrics                             ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
""")

print("🎉 Congratulations! Your aerial threat detection model is ready!")
print("🎯 Use this model to distinguish soldiers from civilians in aerial drone imagery!")

☁️ Uploading results to Google Drive...
✅ Uploaded best.pt (6.0 MB)
✅ Uploaded last.pt (6.0 MB)
✅ Uploaded results.png
✅ Uploaded results.csv
✅ Uploaded confusion_matrix.png
✅ Uploaded confusion_matrix_normalized.png
✅ Uploaded BoxF1_curve.png
✅ Uploaded BoxPR_curve.png
✅ Uploaded BoxP_curve.png
✅ Uploaded BoxR_curve.png
✅ Uploaded labels.jpg
✅ Uploaded val_batch0_labels.jpg
✅ Uploaded val_batch0_pred.jpg
✅ Uploaded val_batch1_labels.jpg
✅ Uploaded val_batch1_pred.jpg
✅ Uploaded val_batch2_labels.jpg
✅ Uploaded val_batch2_pred.jpg
✅ Uploaded train_batch0.jpg
✅ Uploaded train_batch1.jpg
✅ Uploaded train_batch2.jpg
✅ Uploaded args.yaml
✅ Uploaded dataset configuration

📁 All files uploaded to: /content/drive/MyDrive/csc-final-project-aerial.v1-csc_dataset.yolov8/trained_models
🔗 Access your trained models directly from Google Drive!

📥 Files ready for download:
  ✅ runs/train/custom_aerial_detection/weights/best.pt (6.0 MB)
  ✅ runs/train/custom_aerial_detection/weights/last.pt (6.0 MB)


# DOWNLOAD THE WHOLE LOCAL DATASET

In [ ]:
import shutil
from google.colab import files

# Create a zip archive of the civilian_soldier_working folder
shutil.make_archive('/content/civilian_soldier_working', 'zip', '/content/civilian_soldier_working')

# Download the zip file
files.download('/content/civilian_soldier_working.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>